In [ ]:
if True:
    %matplotlib notebook
elif False:
    %matplotlib inline
else:
    %matplotlib qt

In [ ]:
import sys
import os
import getpass
import numpy as np
import matplotlib.pylab as plt

In [ ]:
import pyelegant as pe
from pyelegant import elebuilder
from pyelegant import remote
from pyelegant import sdds
from pyelegant import util

In [ ]:
E_MeV = 3e3

#ini_LTE_filepath = 'nsls2cb72pm.lte'
#ini_LTE_filepath = 'nsls2cb72pm_LS_SS_LS.lte'
ini_LTE_filepath = 'nsls2cb72pm_LS_SS_LS_ZeroSexts.lte'

use_beamline = 'CELL_LS_SS_LS'

nsls2_flr_filepath = '/GPFS/APC/yhidaka/git_repos/nsls2cb/nsls2.flr'

In [ ]:
output_filepath = f'{ini_LTE_filepath}.twi.hdf5'

pe.calc_ring_twiss(output_filepath, ini_LTE_filepath, E_MeV, use_beamline=use_beamline, 
                   radiation_integrals=True, element_divisions=10)

In [ ]:
pe.plot_twiss(
    output_filepath, print_scalars=['ex0', 'Jx', 'nux', 'nuy', 'dnux/dp', 'dnuy/dp', 'alphac'],
    disp_elem_names=dict(bends=True, quads=True))

# First create an ELE file to check the initial lattice

In [ ]:
ed = elebuilder.EleDesigner()

ed.add_block('run_setup',
    lattice = ini_LTE_filepath, p_central_mev = 3e3, 
    use_beamline=use_beamline, parameters='%s.param')

ed.add_newline()

ed.add_block('twiss_output', filename = '%s.twi', matched = True, radiation_integrals = True)

ed.add_newline()

ed.add_block('floor_coordinates', filename = '%s.flr')

ed.add_newline()

# The following are required to generate "%s.param"
ed.add_block('run_control')
ed.add_block('bunched_beam')
ed.add_block('track')

ed.write()

In [ ]:
ed.blocks.get_avail_blocks()

In [ ]:
print(ed.blocks.get_default_str('run_setup'))

In [ ]:
pe.run(ed.ele_filepath, print_stdout=True, print_stderr=True)

In [ ]:
ini_res = ed.load_sdds_output_files()['data']

In [ ]:
list(ini_res)

In [ ]:
ini_res['param']

# Start building Optimization ELE file

In [ ]:
ed.clear()

ed.add_block('run_setup',
    lattice = ini_LTE_filepath, # If you change this LTE file, make sure to change the LTE file
    # name to the same file in the "run_setup" block towards the end.
    p_central_mev = 3e3, use_beamline=use_beamline, semaphore_file = '%s.done',
    parameters='%s.param', default_order = 2)

In [ ]:
ed.set_bookmark('load_param')

In [ ]:
ed.add_newline()
ed.add_comment('! Ask for twiss parameters during optimization')
ed.add_block('twiss_output', 
    output_at_each_step = True, matched = True, radiation_integrals = True, concat_order = 2)

ed.add_newline()
ed.add_comment('! Ask for floor coordinates (during optimization)')
ed.add_block('floor_coordinates', filename = '%s.flr')

ed.add_newline()
ed.add_comment('! Load floor coordinate data for NSLS-II')
ed.add_comment('! - First ID')
ed.add_block('rpn_load',
    filename = nsls2_flr_filepath, tag = 'flr1', match_column = 'ElementName', 
    match_column_value = 'MID', matching_row_number = 0)
ed.add_comment('! - Second ID')
ed.add_block('rpn_load',
    filename = nsls2_flr_filepath, tag = 'flr2', match_column = 'ElementName',
    match_column_value = 'MID', matching_row_number = 1)

ed.add_newline()
ed.add_block('run_control')

In [ ]:
ed.print_last_block()

In [ ]:
ed.print_whole()

# Optimization Setup

In [ ]:
ed.add_newline()
ed.add_comment('! Use parallel hybrid simplex optimization')

ed.add_block('parallel_optimization_setup', 
    mode = 'minimize', method = 'hybridsimplex', hybrid_simplex_comparison_interval = 100,
    #target = 1e-06, tolerance = 1e-14, n_passes = 3, n_evaluations = 1500, n_restarts = 50,
    #target = 1e-06, tolerance = 1e-14, n_passes = 3, n_evaluations = 200, n_restarts = 10,
    target = 1e-06, tolerance = 1e-14, n_passes = 3, n_evaluations = 100, n_restarts = 5,
    verbose = False, log_file = '/dev/tty', output_sparsing_factor = 100, 
    term_log_file = '%s.tlog', simplex_log = '%s.simlog',
    simplex_log_interval = 50,
    # ^ Depending on your filesystem, you may want to increase this in order to improve performance
)

# Optimization Variables

## # Knobs
- K1: 'BD', 'BF', 'LBD', 'LBF', 'HBD', 'HBF', 'MBD', 'MBF', 'QL1', 'QL2', 'QH1', 'QH2', 'QM1', 'QM2'
- L: 'DL5', 'DH5'
- ANGLE: 'BD', 'BF', 'LBD', 'LBF', 'HBD', 'HBF', 'MBD', 'MBF'

In [ ]:
ed.add_newline()
ed.add_comment('! Vary focusing gradients')

for name in ['BD', 'BF', 'LBD', 'LBF', 'HBD', 'HBF', 'MBD', 'MBF']:
    ed.add_block('optimization_variable',
        name = name, item = 'K1', lower_limit=-20, upper_limit=20, step_size=1e-4)
    
for name in ['QL1', 'QL2', 'QH1', 'QH2', 'QM1', 'QM2']:
    ed.add_block('optimization_variable',
        name = name, item = 'K1', lower_limit=-7, upper_limit=7, step_size=1e-4)

In [ ]:
# Initial values are checked to see whether they fall between lower and upper limit

if False:
    ed.add_newline()
    ed.add_comment('! Allow straight section lengths to vary (helps matching floor coordinates)')

    ed.add_block('optimization_variable',
        name = 'DL5', item = 'L', lower_limit = 3.2, upper_limit = 3.4)
    ed.add_block('optimization_variable',
        name = 'DH5', item = 'L', lower_limit = 4.4, upper_limit = 4.6)
else:
    ed.add_newline()
    ed.add_comment('! Allow straight section lengths to vary (helps matching floor coordinates)')

    ed.add_block('optimization_variable',
        name = 'DL5', item = 'L', lower_limit = 3.2, upper_limit = 4.65)
    ed.add_block('optimization_variable',
        name = 'DH5', item = 'L', lower_limit = 3.2, upper_limit = 4.65)

In [ ]:
ed.add_newline()
ed.add_comment('! Dipole Angles')

for name in ['BD', 'BF', 'LBD', 'LBF', 'HBD', 'HBF', 'MBD']:
    ed.add_block('optimization_variable',
        name = name, item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5)

# Optimization Co-Variables

Digression on RPN (Reverse Polish Notation a.k.a. Post-fix Notation), as opposed to the infix notation (i.e., the normal way we write equations).

Consider the following random expression:

\begin{align}
& \beta_{x0} - \alpha_{x0} (s_2 + s_1) + 
    \frac{1 + \alpha_{x0}^2}{3 \beta_{x0}} (s_1^2 + s_1 s_2 + s_2^2) \\
\end{align}

In [ ]:
some_algeb_eq = elebuilder.InfixEquation(
    'betax0 - alphax0 * (s2 + s1) + (1 + alphax0**2) / (3 * betax0) * (s1**2 + s1*s2 + s2**2)')
some_algeb_eq

As long as a valid Python expression string is passed in the previous cell,
corresponding RPN expression is given by:

In [ ]:
some_algeb_eq.torpn()

If you give an invalid (infix) expression, you will get a warning:

In [ ]:
some_algeb_eq = elebuilder.InfixEquation(
    'betax0 - alphax0 * s2 + s1) + (1 + alphax0**2) / (3 * betax0) * (s1**2 + s1*s2 + s2**2)')

In [ ]:
c = elebuilder.RPNCalculator()

In [ ]:
d = {
    'betax0': 1.5,
    'alphax0': -20.0,
    's1': 0.1, 
    's2': 0.5,
}

c.clear_buffer()
c.get_buffer(
    'betax0 alphax0 s2 s1 + * - 1 alphax0 2 pow + 3 betax0 * / s1 2 pow s1 s2 * + s2 2 pow + * +', d)

In [ ]:
rf = ed.rpnfuncs

In [ ]:
[_s for _s in dir(rf) if not _s.startswith('_')]

In [ ]:
help(rf.segt)

In [ ]:
rf.segt?

In [ ]:
help(rf.dtor)

In [ ]:
help(rf.maxn)

In [ ]:
optim_term_rpnvars = ed.get_rpn_vars('optimization_term')
rd = optim_term_rpnvars.dict
rv = optim_term_rpnvars.namespace

In [ ]:
list(rd)

In [ ]:
rd['BD.ANGLE']

In [ ]:
type(rd['BD.ANGLE'])

In [ ]:
# optim_term_rpnvars = ed.get_rpn_vars('optimization_term')
# rd = optim_term_rpnvars.dict
# rv = optim_term_rpnvars.namespace

(rv.BD__DOT__ANGLE) is (rd['BD.ANGLE'])

In [ ]:
ring_multiplier = 15

In [ ]:
angle_sum_rad = 0.0
for bend_name in ['BD', 'BF', 'LBD', 'LBF', 'HBD', 'HBF', 'MBD', 'MBF']:
    angle_sum_rad += ed.get_LTE_elem_count(bend_name) * ed.get_LTE_elem_prop(bend_name, 'ANGLE')

print('{:.6f} [deg] for this lattice; {:.6f} [deg] for full ring'.format(
    np.rad2deg(angle_sum_rad), np.rad2deg(angle_sum_rad) * ring_multiplier))

In [ ]:
covar_rpnvars = ed.get_rpn_vars('optimization_covariable')
rd_co = covar_rpnvars.dict

# "rd_co" dict contains variables that are available for use in "&optimization_covariable" block.
# "rd" dict contains variables that are available for use in "&optimization_term" block.

In [ ]:
# When this cell is run after sequentially running all the cells above,
# it should generate a key error, as "MBF.ANGLE" has not been yet declared with
# "&optimization_variable", and so the variable is not yet available to be used
# in "&optimization_covariable" block.

rf.dtor(24) - (
    rd_co['BD.ANGLE'] * ed.get_LTE_elem_count('BD') + 
    rd_co['BF.ANGLE'] * ed.get_LTE_elem_count('BF') +
    rd_co['LBD.ANGLE'] * ed.get_LTE_elem_count('LBD') +
    rd_co['LBF.ANGLE'] * ed.get_LTE_elem_count('LBF') +
    rd_co['HBD.ANGLE'] * ed.get_LTE_elem_count('HBD') +
    rd_co['HBF.ANGLE'] * ed.get_LTE_elem_count('HBF') +
    rd_co['MBD.ANGLE'] * ed.get_LTE_elem_count('MBD') +
    rd_co['MBF.ANGLE'] * ed.get_LTE_elem_count('MBF')
)

In [ ]:
# When this cell is run after sequentially running all the cells above,
# it should generate a key error, as "MBF.ANGLE" has not been yet declared with
# "&optimization_variable", and so the variable is not yet available to be used
# in "&optimization_term" block.

rd['MBF.ANGLE']

In [ ]:
rest_of_bend_angles = (
    rd_co['BD.ANGLE'] * ed.get_LTE_elem_count('BD') + 
    rd_co['BF.ANGLE'] * ed.get_LTE_elem_count('BF') +
    rd_co['LBD.ANGLE'] * ed.get_LTE_elem_count('LBD') +
    rd_co['LBF.ANGLE'] * ed.get_LTE_elem_count('LBF') +
    rd_co['HBD.ANGLE'] * ed.get_LTE_elem_count('HBD') +
    rd_co['HBF.ANGLE'] * ed.get_LTE_elem_count('HBF') +
    rd_co['MBD.ANGLE'] * ed.get_LTE_elem_count('MBD')
)
MBF_angle_infix_eq = (rf.dtor(24) - rest_of_bend_angles) / ed.get_LTE_elem_count('MBF')

print(MBF_angle_infix_eq)

In [ ]:
rpn_str = MBF_angle_infix_eq.torpn()

rpn_str

In [ ]:
ed.add_newline()
ed.add_comment('! Set MBF angle so that the total angle of this lattice becomes 24 degrees')

if True:
    ed.add_block('optimization_covariable',
        name = 'MBF', item = 'ANGLE', equation = MBF_angle_infix_eq)
else:
    ed.add_block('optimization_covariable',
        name = 'MBF', item = 'ANGLE', equation = rpn_str)

In [ ]:
# When this cell is run after sequentially running all the cells above,
# it should NOT generate a key error any more, as "MBF.ANGLE" has been declared with
# "&optimization_covariable", and so the variable is available to be used
# in "&optimization_term" block.

rd['MBF.ANGLE']

In [ ]:
# When this cell is run after sequentially running all the cells above,
# it should still generate a key error, as "MBF.ANGLE" has been declared with 
# "&optimization_covariable", NOT "&optimization_variable". This will make the 
# variable available in "&optimization_term" block, but NOT in "&optimization_covariable."

rd_co['MBF.ANGLE']

# Optimization Terms

In [ ]:
ed.add_newline()
ed.add_comment('! Constrain MBF angle within a reasonable range')

if False:
    ed.add_block('optimization_term',
        term = rf.selt(rd['MBF.ANGLE'], 0.01, 1e-6)
    )

ed.add_block('optimization_term',
    term = rf.segt(rd['MBF.ANGLE'], 0.03, 1e-6)
)

In [ ]:
ed.add_newline()
ed.add_comment('! Natural chromaticity should be minimized')

ed.add_block('optimization_term', 
    term = rf.abs(rd['dnux/dp'] * ring_multiplier) / 10)

ed.add_block('optimization_term', 
    term = rf.abs(rd['dnuy/dp'] * ring_multiplier) / 10)

In [ ]:
ed.add_newline()
ed.add_comment('! Want Jx:[1, 2]')

ed.add_block('optimization_term', term = rf.selt(rd['Jx'], 1, 1e-3))
ed.add_block('optimization_term', term = rf.segt(rd['Jx'], 2, 1e-3))

In [ ]:
ed.add_newline()
ed.add_comment('! Want etax to be zero in both ID straights')

ed.add_block('optimization_term', term = rf.sene(rd['M_LS#2.etax'], 0, 1e-4))
ed.add_block('optimization_term', term = rf.sene(rd['M_SS#1.etax'], 0, 1e-4))

In [ ]:
ed.add_newline()
ed.add_comment('! Want etax>0.08 in high-dispersion region (weak requirement)')

ed.add_block('optimization_term', term = rf.selt(rd['MDISP#1.etax'], 0.1, 0.01))
ed.add_block('optimization_term', term = rf.selt(rd['MDISP#2.etax'], 0.1, 0.01))

In [ ]:
ed.add_newline()
ed.add_comment('! Minimize the emittance')

eps_x_pm = rd['ex0'] * 1e12
ed.add_block('optimization_term', term = rf.segt(eps_x_pm, 70.0, 1.0) * 10)

In [ ]:
ed.add_newline()
ed.add_comment('! Maximize the momentum compaction')

alphac_x_1e4 = rd['alphac'] * 1e4
ed.add_block('optimization_term', term = rf.selt(alphac_x_1e4, 3.6, 0.1))

In [ ]:
if False:
    ed.add_newline()
    ed.add_comment('! Try to make betax=betay=L/2 in IDs, where L is the *total* straight length')
    
    ed.add_block('optimization_term', term = rf.sene(rd['M_SS#1.betax'] / rd['DL5.L'], 1, 0.1) * 1e2)
    ed.add_block('optimization_term', term = rf.sene(rd['M_SS#1.betay'] / rd['DL5.L'], 1, 0.1) * 1e2)
    ed.add_block('optimization_term', term = rf.sene(rd['M_LS#2.betax'] / rd['DH5.L'], 1, 0.1) * 1e2)
    ed.add_block('optimization_term', term = rf.sene(rd['M_LS#2.betay'] / rd['DH5.L'], 1, 0.1) * 1e2)
else:
    ed.add_newline()
    ed.add_comment('! Try to make betax & betay the same as NSLS-II Day-1 bare')
    
    ed.add_block('optimization_term', term = rf.sene(rd['M_SS#1.betax'], 1.846, 0.2) * 5)
    ed.add_block('optimization_term', term = rf.sene(rd['M_SS#1.betay'], 1.171, 0.1) * 5)
    ed.add_block('optimization_term', term = rf.sene(rd['M_LS#2.betax'], 20.466, 2.0) * 5)
    ed.add_block('optimization_term', term = rf.sene(rd['M_LS#2.betay'], 3.369, 0.3) * 5)

## Digression: Floor Coordinates

In [ ]:
flr_data = sdds.sdds2dicts(nsls2_flr_filepath)[0]
N2_X = flr_data['columns']['X']
N2_Z = flr_data['columns']['Z']
N2_ElemNames = flr_data['columns']['ElementName']
N2_SS_index, N2_LS_index = np.where(N2_ElemNames == 'MID')[0][:2]

In [ ]:
SS_index = np.where(ini_res['flr']['columns']['ElementName'] == 'M_SS')[0][0]
LS_index = np.where(ini_res['flr']['columns']['ElementName'] == 'M_LS')[0][1] # Must pick the 2nd index

In [ ]:
plt.figure()
plt.plot(N2_X[:N2_LS_index], N2_Z[:N2_LS_index], 'b.-')
plt.plot(N2_X[N2_SS_index], N2_Z[N2_SS_index], 'k+', ms=20)
plt.plot(N2_X[N2_LS_index], N2_Z[N2_LS_index], 'k+', ms=20)
plt.plot(ini_res['flr']['columns']['X'][SS_index], ini_res['flr']['columns']['Z'][SS_index], 'ro') 
plt.plot(ini_res['flr']['columns']['X'][LS_index], ini_res['flr']['columns']['Z'][LS_index], 'ro') 
plt.grid(True)
plt.tight_layout()

## Digression: Floor Coordinates (end)

In [ ]:
# (M_SS#1 in this LTE file) == (MID#1 in nsls2.flr) => flr1.X & flr1.Z
# (M_LS#2 in this LTE file) == (MID#2 in nsls2.flr) => flr2.X & flr2.Z

ed.add_newline()
ed.add_comment('! Keep the radius within 1 mm of NSLS-II')

eq = rf.sene(rf.sqrt(rf.sqr(rd['M_SS#1.X']) + rf.sqr(rd['M_SS#1.Z'])), 
             rf.sqrt(rf.sqr(rd['flr1.X']) + rf.sqr(rd['flr1.Z'])), 
             1e-3)

ed.add_block('optimization_term', term = eq)

eq = rf.sene(rf.sqrt(rf.sqr(rd['M_LS#2.X']) + rf.sqr(rd['M_LS#2.Z'])), 
             rf.sqrt(rf.sqr(rd['flr2.X']) + rf.sqr(rd['flr2.Z'])), 
             1e-3)

ed.add_block('optimization_term', term = eq)

In [ ]:
ed.add_newline()
ed.add_comment('! Make sure beta is not too large anywhere')

ed.add_block('optimization_term', term = rf.segt(rd['max.betax'], 30.0, 1.0))
ed.add_block('optimization_term', term = rf.segt(rd['max.betay'], 30.0, 1.0))

In [ ]:
ed.add_newline()
ed.add_comment('! Make sure beta is not too small anywhere')

ed.add_block('optimization_term', term = rf.selt(rd['min.betax'], 0.2, 0.1))
ed.add_block('optimization_term', term = rf.selt(rd['min.betay'], 0.2, 0.1))

In [ ]:
ed.add_newline()
ed.add_comment("! Ensure that central particle isn't lost")

ed.add_block('optimization_term', term = rf.sene(rd['Particles'], 1, 1e-10))

# Add more blocks required to run optimizer

In [ ]:
ed.add_newline()
ed.add_comment("! Beam consists of central particle only")

ed.add_block('bunched_beam')

In [ ]:
ed.add_newline()
ed.add_comment("! Start optimization")

ed.add_block('optimize')

# Add blocks for final evaluation of optimized lattice

In [ ]:
ed.add_newline()
ed.add_comment("! Evaluate the results of optimization")

ed.add_newline()

ed.add_block('run_setup',
    lattice = ini_LTE_filepath,
    use_beamline='RING_LS_SS',
    # ^ Note that here I am using the full ring, not 2 ring cells
    p_central_mev = 3e3, semaphore_file = '%s.done', magnets = '%s.mag', default_order = 2,
)

ed.add_newline()

ed.add_block('load_parameters',filename = '%s.param', change_defined_values = True)

ed.add_newline()

ed.add_block('twiss_output',
    filename = '%s.twi', matched = True, radiation_integrals = True, concat_order = 2)

ed.add_newline()

ed.add_block('floor_coordinates', filename = '%s.flr')

# Save Optimized Lattice into a New LTE file

In [ ]:
ed.add_newline()

ed.add_block('save_lattice', filename = '%s.newlte')

# Finally write the contents into a ELE file

In [ ]:
ed.write()

print(f'ELE contents has been written into "{ed.ele_filepath}"')
print(' ')
print('Running ELEGANT with this ELE file should generate following output files:')
for fp in ed.actual_output_filepath_list:
    print(f'   {fp}')

# Start running the optimizer

In [ ]:
remote.start_hybrid_simplex_optimizer(
    ed, ntasks=18, partition='short', job_name='job', time_limit='7:00', 
    email_end=True, email_address=f'{getpass.getuser()}@bnl.gov',
    show_progress_plot=True)

In [ ]:
print('Consolidating SDDS data files...')
_d = ed.load_sdds_output_files()
output, meta = _d['data'], _d['meta']
print('Finished.')

In [ ]:
util.pprint_optim_term_log(output['tlog'])

In [ ]:
# Save results into a HDF5 file
print('Writing data to HDF5 file...')
sys.stdout.flush()
output_filepath = 'hybrid_simplex_results.hdf5'
util.robust_sdds_hdf5_write(
    output_filepath, [output, meta], nMaxTry=10, sleep=10.0)
print('Finished.')
sys.stdout.flush()

In [ ]:
# Save the dictionaries into a gzipped pickle file
print('Writing data to pgz file...')
sys.stdout.flush()
output_filepath = 'hybrid_simplex_results.pgz'
mod_output = {}
for k, v in output.items():
    mod_output[k] = {}
    if 'params' in v:
        mod_output[k]['scalars'] = v['params']
    if 'columns' in v:
        mod_output[k]['arrays'] = v['columns']
mod_meta = {}
for k, v in meta.items():
    mod_meta[k] = {}
    if 'params' in v:
        mod_meta[k]['scalars'] = v['params']
    if 'columns' in v:
        mod_meta[k]['arrays'] = v['columns']
util.robust_pgz_file_write(
    output_filepath, [mod_output, mod_meta], nMaxTry=10, sleep=10.0)
print('Finished.')
sys.stdout.flush()

In [ ]:
fin_LTE_filepath = [fp for fp in ed.actual_output_filepath_list if fp.endswith('.newlte')][0]
output_filepath = f'{fin_LTE_filepath}.twi.hdf5'

pe.calc_ring_twiss(output_filepath, fin_LTE_filepath, E_MeV, use_beamline=use_beamline, 
                   radiation_integrals=True, element_divisions=10)

In [ ]:
pe.plot_twiss(
    output_filepath, print_scalars=['ex0', 'Jx', 'nux', 'nuy', 'dnux/dp', 'dnuy/dp', 'alphac'])

# Now try to resume optimization from where it ended

In [ ]:
# Move the pointer of the "EleDesigner" object to the previously bookmarked location to 
# insert a new "&load_parameters" block

ed.seek_bookmark('load_param')

In [ ]:
param_filepath = [fp for fp in ed.actual_output_filepath_list if fp.endswith('.param')][0]
param_filepath

In [ ]:
ed.add_newline()
ed.add_comment('! Load solution from previous optimization')
ed.add_block('load_parameters', filename = param_filepath, change_defined_values = True)

In [ ]:
ed.print_whole()

In [ ]:
ed.write()

In [ ]:
remote.start_hybrid_simplex_optimizer(
    ed, ntasks=18, partition='short', job_name='job', time_limit='7:00', 
    email_end=False, email_address=f'{getpass.getuser()}@bnl.gov',
    show_progress_plot=True)

In [ ]:
print('Consolidating SDDS data files...')
_d = ed.load_sdds_output_files()
output, meta = _d['data'], _d['meta']
print('Finished.')

In [ ]:
util.pprint_optim_term_log(output['tlog'])

In [ ]:
# Delete the raw SDDS files
ed.delete_temp_files()

In [ ]:
# Delete the temporary ELE file
ed.delete_ele_file()

# Phase Cancellation of First-order Geometric Terms

From Eq. (97) of J. Bengtsson, "The Sextupole Scheme for the Swiss Light Source (SLS): An Analytic Approach," SLS Note 9/97:

\begin{align}
h_{21000} &= h_{12000}^* = -\frac{1}{8} \sum_{i=1}^{N} (b_{3i} L) \beta_{xi}^{3/2} e^{i \mu_{xi}} \\
h_{30000} &= h_{03000}^* = -\frac{1}{24} \sum_{i=1}^{N} (b_{3i} L) \beta_{xi}^{3/2} e^{i 3 \mu_{xi}} \\
h_{10110} &= h_{01110}^* = \frac{1}{4} \sum_{i=1}^{N} (b_{3i} L) \beta_{xi}^{1/2} \beta_{yi} e^{i \mu_{xi}} \\
h_{10020} &= h_{01200}^* = \frac{1}{8} \sum_{i=1}^{N} (b_{3i} L) \beta_{xi}^{1/2} \beta_{yi} e^{i (\mu_{xi} - 2\mu_{yi})} \\
h_{10200} &= h_{01020}^* = \frac{1}{8} \sum_{i=1}^{N} (b_{3i} L) \beta_{xi}^{1/2} \beta_{yi} e^{i (\mu_{xi} + 2\mu_{yi})} \\
\end{align}

When the following conditions are satisfied, all of these driving terms vanish:

\begin{align}
&b_{3i} L = b_{3j} L \\
&\beta_{xi} = \beta_{xj}, \beta_{yi} = \beta_{yj} \\
&\begin{cases}
  \mu_{xi} = \mu_{xj} + (2 m + 1) \pi \\
  \mu_{yi} = \mu_{yj} + m \pi \\
\end{cases} \, \mathrm{or} \, 
\begin{cases}
  |\nu_{xi} - \nu_{xj}| = m + 0.5 \\
  \begin{cases}
    |\nu_{yi} - \nu_{yj}| = m \\
    |\nu_{yi} - \nu_{yj}| = m + 0.5 \\
  \end{cases}
\end{cases}
\end{align}

where $m = 0, 1, 2, ...$. If we use the fractional normalized phase differences $\delta \nu_x, \delta \nu_y$ as defined by

\begin{align}
  &\Delta \nu_x := \nu_{xj} - \nu_{xi}, \Delta \nu_y := \nu_{yj} - \nu_{yi} \\
  &\delta \nu_x := \Delta \nu_x - \mathrm{floor}(\Delta \nu_x), \delta \nu_x := \Delta \nu_y - \mathrm{floor}(\Delta \nu_y) \\
\end{align}

we can re-write the phase relation matching condition as the numerical minimiation problem of the following 2 terms:

\begin{cases}
  |\delta \nu_x - 0.5| \\
  \min( {\delta \nu_y, |\delta \nu_y - 0.5|, 1 - \delta \nu_y} ) \\
\end{cases}


In [ ]:
pe.plot_twiss(output_filepath, print_scalars=[])

In [ ]:
# When this cell is run after sequentially running all the cells above,
# it should generate a key error "MDISP#1.psix".

nux_DA_LS_DA = rd['MDISP#1.psix'] / (2 * rd['pi']) * 2
frac_nux_DA_LS_DA = nux_DA_LS_DA - rf.floor(nux_DA_LS_DA)
v2 = rf.abs(frac_nux_DA_LS_DA - 0.5)
print(rf.segt(v2 * 1e3, 2.0, 1.0).torpn())

In [ ]:
# The error in the previous cell happened because we moved the "EleDesigner" object's 
# pointer to the "load_param" bookmark in order to insert the "&load_parameters" block.
# At that point, "MDISP#1.*" variables are not yet available. So, now we move back
# the pointer to the bottom.

ed.seek_bookmark('bottom')

In [ ]:
# Now running this cell (exactly the same cell as the one 2 cells above should
# run successfully.

nux_DA_LS_DA = rd['MDISP#1.psix'] / (2 * rd['pi']) * 2
frac_nux_DA_LS_DA = nux_DA_LS_DA - rf.floor(nux_DA_LS_DA)
v2 = rf.abs(frac_nux_DA_LS_DA - 0.5)
print(rf.segt(v2 * 1e3, 2.0, 1.0).torpn())

In [ ]:
nuy_DA_LS_DA = rd['MDISP#1.psiy'] / (2 * rd['pi']) * 2
frac_nuy_DA_LS_DA = nuy_DA_LS_DA - rf.floor(nuy_DA_LS_DA)
v1 = frac_nuy_DA_LS_DA
v2 = rf.abs(frac_nuy_DA_LS_DA - 0.5)
v3 = 1.0 - frac_nuy_DA_LS_DA
print(rf.segt(rf.minn(v1, v2, v3) * 1e3, 2.0, 1.0).torpn())

In [ ]:
eq_dphix = elebuilder.OptimizationTerm(ed)

In [ ]:
sorted([s for s in list(rd) if s.startswith('nu')])

In [ ]:
eq_dphix.assign('nux_DA_LS_DA', rd['MDISP#1.psix'] / (2 * rd['pi']) * 2)

In [ ]:
sorted([s for s in list(rd) if s.startswith('nu')])

In [ ]:
eq_dphix.assign('frac_nux_DA_LS_DA', rd['nux_DA_LS_DA'] - rf.floor(rd['nux_DA_LS_DA']))

In [ ]:
sorted([s for s in list(rd) if s.startswith('frac')])

In [ ]:
eq_dphix.assign('v2', rf.abs(rd['frac_nux_DA_LS_DA'] - 0.5))

In [ ]:
eq_dphix.set_final(rf.segt(rd['v2'] * 1e3, 2.0, 1.0))

In [ ]:
ed.add_newline()
ed.add_comment('! LS dnux condition')

ed.add_block('optimization_term', term = eq_dphix)

In [ ]:
eq_dphiy = elebuilder.OptimizationTerm(ed)
eq_dphiy.assign('nuy_DA_LS_DA', rd['MDISP#1.psiy'] / (2 * rd['pi']) * 2)
eq_dphiy.assign('frac_nuy_DA_LS_DA', rd['nuy_DA_LS_DA'] - rf.floor(rd['nuy_DA_LS_DA']))

In [ ]:
eq_dphiy.assign('v1', rd['frac_nuy_DA_LS_DA'])
eq_dphiy.assign('v2', rf.abs(rd['frac_nuy_DA_LS_DA'] - 0.5))
eq_dphiy.assign('v3', 1.0 - rd['frac_nuy_DA_LS_DA'])

In [ ]:
eq_dphiy.set_final(rf.segt(rf.minn(rd['v1'], rd['v2'], rd['v3']) * 1e3, 2.0, 1.0))

In [ ]:
ed.add_newline()
ed.add_comment('! LS dnuy condition')

ed.add_block('optimization_term', term = eq_dphiy)

In [ ]:
with elebuilder.OptimizationTerm(ed) as eq:
    eq.assign('nux_DA_SS_DA', (rd['MDISP#2.psix'] - rd['MDISP#1.psix']) / (2 * rd['pi']) * 2)
    eq.assign('frac_nux_DA_SS_DA', rd['nux_DA_SS_DA'] - rf.floor(rd['nux_DA_SS_DA']))

    eq.assign('v2', rf.abs(rd['frac_nux_DA_SS_DA'] - 0.5))
    
    eq.set_final(rf.segt(rd['v2'] * 1e3, 2.0, 1.0))

ed.add_newline()
ed.add_comment('! SS dnux condition')

ed.add_block('optimization_term', term = eq)

In [ ]:
with elebuilder.OptimizationTerm(ed) as eq:
    eq.assign('nuy_DA_SS_DA', (rd['MDISP#2.psiy'] - rd['MDISP#1.psiy']) / (2 * rd['pi']) * 2)
    eq.assign('frac_nuy_DA_SS_DA', rd['nuy_DA_SS_DA'] - rf.floor(rd['nuy_DA_SS_DA']))
    
    eq.assign('v1', rd['frac_nuy_DA_SS_DA'])
    eq.assign('v2', rf.abs(rd['frac_nuy_DA_SS_DA'] - 0.5))
    eq.assign('v3', 1.0 - rd['frac_nuy_DA_SS_DA'])    
    
    eq.set_final(rf.segt(rf.minn(rd['v1'], rd['v2'], rd['v3']) * 1e3, 2.0, 1.0))

ed.add_newline()
ed.add_comment('! SS dnuy condition')

ed.add_block('optimization_term', term = eq)

In [ ]:
with elebuilder.OptimizationTerm(ed, intermediate=True) as eq:
    eq.assign('nux_DA_LS_DA', rd['MDISP#1.psix'] / (2 * rd['pi']) * 2)
    eq.assign('frac_nux_DA_LS_DA', rd['nux_DA_LS_DA'] - rf.floor(rd['nux_DA_LS_DA']))

    eq.assign('v2', rf.abs(rd['frac_nux_DA_LS_DA'] - 0.5))
    
    eq.assign('DA_LS_DA_dnux', rf.segt(rd['v2'] * 1e3, 2.0, 1.0))

ed.add_newline()
ed.add_comment('! Intermediate LS dnux condition')

ed.add_block('optimization_term', term = eq)

In [ ]:
with elebuilder.OptimizationTerm(ed, intermediate=True) as eq:
    eq.assign('nuy_DA_LS_DA', rd['MDISP#1.psiy'] / (2 * rd['pi']) * 2)
    eq.assign('frac_nuy_DA_LS_DA', rd['nuy_DA_LS_DA'] - rf.floor(rd['nuy_DA_LS_DA']))
    
    eq.assign('v1', rd['frac_nuy_DA_LS_DA'])
    eq.assign('v2', rf.abs(rd['frac_nuy_DA_LS_DA'] - 0.5))
    eq.assign('v3', 1.0 - rd['frac_nuy_DA_LS_DA'])
    
    eq.assign('DA_LS_DA_dnuy', rf.segt(rf.minn(rd['v1'], rd['v2'], rd['v3']) * 1e3, 2.0, 1.0))

ed.add_newline()
ed.add_comment('! Intermediate LS dnuy condition')

ed.add_block('optimization_term', term = eq)

In [ ]:
with elebuilder.OptimizationTerm(ed, intermediate=True) as eq:
    eq.assign('nux_DA_SS_DA', (rd['MDISP#2.psix'] - rd['MDISP#1.psix']) / (2 * rd['pi']) * 2)
    eq.assign('frac_nux_DA_SS_DA', rd['nux_DA_SS_DA'] - rf.floor(rd['nux_DA_SS_DA']))

    eq.assign('v2', rf.abs(rd['frac_nux_DA_SS_DA'] - 0.5))
    
    eq.assign('DA_SS_DA_dnux', rf.segt(rd['v2'] * 1e3, 2.0, 1.0))

ed.add_newline()
ed.add_comment('! Intermediate SS dnux condition')

ed.add_block('optimization_term', term = eq)

In [ ]:
with elebuilder.OptimizationTerm(ed, intermediate=True) as eq:
    eq.assign('nuy_DA_SS_DA', (rd['MDISP#2.psiy'] - rd['MDISP#1.psiy']) / (2 * rd['pi']) * 2)
    eq.assign('frac_nuy_DA_SS_DA', rd['nuy_DA_SS_DA'] - rf.floor(rd['nuy_DA_SS_DA']))
    
    eq.assign('v1', rd['frac_nuy_DA_SS_DA'])
    eq.assign('v2', rf.abs(rd['frac_nuy_DA_SS_DA'] - 0.5))
    eq.assign('v3', 1.0 - rd['frac_nuy_DA_SS_DA'])
    
    eq.assign('DA_SS_DA_dnuy', rf.segt(rf.minn(rd['v1'], rd['v2'], rd['v3']) * 1e3, 2.0, 1.0))

ed.add_newline()
ed.add_comment('! Intermediate SS dnuy condition')

ed.add_block('optimization_term', term = eq)

In [ ]:
ed.add_newline()
ed.add_comment('! final dnux/dnuy condition')

final_obj = rf.min2(
    rf.max2(rd['DA_LS_DA_dnux'], rd['DA_LS_DA_dnuy']), 
    rf.max2(rd['DA_SS_DA_dnux'], rd['DA_SS_DA_dnuy'])
)

ed.add_block('optimization_term', term = final_obj)